In [1]:
from utils.dataloader import H5Dataset
from utils.EarlyStopping import EarlyStopping

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from sklearn.metrics import classification_report
from tqdm import tqdm

from torchvision.models import ResNet152_Weights

import numpy as np
import os

In [2]:
num_classes = 4
k_folds = 5
num_epochs = 15
batch_size = 32
patience = 2
random_state = 21
checkpoint_folder = "resnet152"


# Caminho para os arquivos h5
h5_files = [
    "h5_files/GunshotForensic_feature.h5",
    "h5_files/HEAR_Train_feature.h5",
    "h5_files/MIVIA_test_feature.h5",
    "h5_files/MIVIA_train_feature.h5",
    "h5_files/SESA_feature.h5",
    "h5_files/vsd_clipped_feature.h5",
    "h5_files/Audioset_feature.h5",
]

# Carregando os dataloaders
dataset = H5Dataset(
    h5_files,
    exclude_classes=[4, 5, 6],
    n_splits=k_folds,
    random_state=random_state,
)
dataloaders = dataset.get_k_fold_data_loaders(batch_size=batch_size, num_workers=4)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Treinamento realizado no dispositivo:", device)

criterion = nn.CrossEntropyLoss()

Treinamento realizado no dispositivo: cuda


In [4]:
# Loop de treinamento
for fold, (train_loader, val_loader) in enumerate(dataloaders):
    print(f"Starting fold {fold + 1}/{len(dataloaders)}")

    # Ajustando a ResNet152
    model = models.resnet152(weights=ResNet152_Weights.DEFAULT)

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    model.conv1 = nn.Conv2d(
        1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
    )
    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

    early_stopping = EarlyStopping(
        patience=patience, verbose=True, fold=fold, checkpoint_folder=checkpoint_folder
    )

    for epoch in range(num_epochs):  # número de épocas
        print(f"Starting epoch {epoch + 1}/{num_epochs}")

        # Treinamento
        model.train()
        running_loss_train = 0.0
        running_corrects_train = 0
        for i, (inputs, labels) in tqdm(
            enumerate(train_loader), total=len(train_loader)
        ):
            inputs = inputs.unsqueeze(1).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects_train += torch.sum(preds == labels.data)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss_train += loss.item() * inputs.size(0)

        epoch_loss_train = running_loss_train / len(train_loader.dataset)
        epoch_acc_train = running_corrects_train.double() / len(train_loader.dataset)

        # Validação
        model.eval()
        running_loss_val = 0.0
        running_corrects_val = 0
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader):
                inputs = inputs.unsqueeze(1).to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                running_corrects_val += torch.sum(preds == labels.data)
                loss = criterion(outputs, labels)

                running_loss_val += loss.item() * inputs.size(0)

                all_labels.extend(labels.data.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        epoch_loss_val = running_loss_val / len(val_loader.dataset)
        epoch_acc_val = running_corrects_val.double() / len(val_loader.dataset)

        print(f"Train Loss: {epoch_loss_train:.4f} Train Acc: {epoch_acc_train:.4f}")
        print(
            f"Validation Loss: {epoch_loss_val:.4f} Validation Acc: {epoch_acc_val:.4f}"
        )

        early_stopping(epoch_loss_val, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    print(f"Finished fold {fold + 1}/{len(dataloaders)}")

print("Finished Training")

Starting fold 1/5
Starting epoch 1/15


100%|██████████| 563/563 [01:17<00:00,  7.24it/s]


Train Loss: 0.8794 Train Acc: 0.5988
Validation Loss: 0.6865 Validation Acc: 0.6909
Validation loss decreased (inf --> 0.686457).  Saving model ...
Starting epoch 2/15


100%|██████████| 563/563 [01:20<00:00,  7.01it/s]


Train Loss: 0.6451 Train Acc: 0.7132
Validation Loss: 0.5822 Validation Acc: 0.7474
Validation loss decreased (0.686457 --> 0.582248).  Saving model ...
Starting epoch 3/15


...

Train Loss: 0.3166 Train Acc: 0.8701
Validation Loss: 0.3370 Validation Acc: 0.8626
Validation loss decreased (0.345962 --> 0.336956).  Saving model ...
Starting epoch 14/15


100%|██████████| 563/563 [01:17<00:00,  7.27it/s]


Train Loss: 0.3040 Train Acc: 0.8748
Validation Loss: 0.3497 Validation Acc: 0.8551
EarlyStopping counter: 1 out of 2
Starting epoch 15/15


100%|██████████| 563/563 [01:22<00:00,  6.85it/s] 

Train Loss: 0.2969 Train Acc: 0.8769
Validation Loss: 0.3594 Validation Acc: 0.8493
EarlyStopping counter: 2 out of 2
Early stopping
Finished fold 5/5
Finished Training


In [5]:
# Dicionário para armazenar os relatórios de classificação para cada fold
classification_reports = []

# Caminho para a pasta com os modelos
model_folder = f"models/{checkpoint_folder}"

# Lista para armazenar a acurácia média de cada fold
accuracy_list = []

# Loop através de cada fold
for fold in range(k_folds):
    # Carregando o modelo
    model_path = os.path.join(model_folder, f"checkpoint_{fold}.pth")
    model.load_state_dict(torch.load(model_path))

    # Pegando o dataloader de teste para este fold
    _, test_loader = dataloaders[fold]

    # Avaliando o modelo neste fold
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.unsqueeze(1).to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    # Calculando a acurácia para este fold
    fold_accuracy = np.mean(np.array(all_labels) == np.array(all_preds))
    accuracy_list.append(fold_accuracy)

    # Gerando o relatório de classificação para este fold
    report = classification_report(all_labels, all_preds, output_dict=True)
    classification_reports.append(report)

# Calculando a média e desvio padrão das métricas para cada classe em todos os folds
metrics_list = {
    class_: {"precision": [], "recall": [], "f1-score": [], "support": []}
    for class_ in range(num_classes)
}

avg_metrics = metrics_list
for report in classification_reports:
    for class_ in range(num_classes):
        metrics_list[class_]["precision"].append(report[str(class_)]["precision"])
        metrics_list[class_]["recall"].append(report[str(class_)]["recall"])
        metrics_list[class_]["f1-score"].append(report[str(class_)]["f1-score"])
        metrics_list[class_]["support"].append(report[str(class_)]["support"])

for class_ in range(num_classes):
    avg_metrics[class_]["precision_std"] = np.std(avg_metrics[class_]["precision"])
    avg_metrics[class_]["recall_std"] = np.std(avg_metrics[class_]["recall"])
    avg_metrics[class_]["f1-score_std"] = np.std(avg_metrics[class_]["f1-score"])
    avg_metrics[class_]["support_std"] = np.round(
        np.std(avg_metrics[class_]["support"])
    ).astype(int)

    avg_metrics[class_]["precision"] = np.mean(metrics_list[class_]["precision"])
    avg_metrics[class_]["recall"] = np.mean(metrics_list[class_]["recall"])
    avg_metrics[class_]["f1-score"] = np.mean(metrics_list[class_]["f1-score"])
    avg_metrics[class_]["support"] = np.round(
        np.mean(metrics_list[class_]["support"])
    ).astype(int)

# Média e desvio padrão da acurácia
accuracy_mean = np.mean(accuracy_list)
accuracy_std = np.std(accuracy_list)


print("Acurácias:", accuracy_list)

Acurácias: [0.8475131981105862, 0.8295081967213115, 0.8416782439566546, 0.8631766144270312, 0.8626208736245415]


In [6]:
def extract_metrics(data):
    metrics = {}

    for fold_index, fold in enumerate(data, start=1):
        for class_label, class_scores in fold.items():
            if class_label.isdigit():
                for metric, value in class_scores.items():
                    metric_key = f"{metric}_classe{class_label}"
                    if metric_key not in metrics:
                        metrics[metric_key] = [value]
                    else:
                        metrics[metric_key].append(value)

    return metrics


import csv


def export_metrics_to_csv(metrics, filename, folder):
    file_path = folder + "/" + filename

    with open(file_path, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        headers = ["Metrica"] + [
            "Fold{}".format(i) for i in range(1, len(metrics[next(iter(metrics))]) + 1)
        ]
        writer.writerow(headers)

        for metric, values in metrics.items():
            row = [metric] + values
            writer.writerow(row)

    print(f"Arquivo CSV exportado com sucesso: {file_path}")

In [7]:
metrics = extract_metrics(classification_reports)
export_metrics_to_csv(metrics, "metrics_resnet152.csv", ".")

Arquivo CSV exportado com sucesso: ./metrics_resnet152.csv


In [10]:
annotation_dict = {
    0: "Nada",
    1: "Grito",
    2: "Violência Física",
    3: "Tiro",
    4: "Explosão",
    5: "Grito de Esforço",
}


print("Acurácia média:", accuracy_mean)
print("Desvio padrão da acurácia:", accuracy_std)

print(
    "================================================================================"
)

for class_ in range(num_classes):
    print(f"Class: {annotation_dict[class_]}")
    print("Precision\t\tRecall\t\t\tF1-Score\t\tSupport")
    print(
        "{:.4f} ± {:.4f}\t\t{:.4f} ± {:.4f}\t\t{:.4f} ± {:.4f}\t\t{}".format(
            avg_metrics[class_]["precision"],
            avg_metrics[class_]["precision_std"],
            avg_metrics[class_]["recall"],
            avg_metrics[class_]["recall_std"],
            avg_metrics[class_]["f1-score"],
            avg_metrics[class_]["f1-score_std"],
            avg_metrics[class_]["support"],
        )
    )
    print(
        "================================================================================"
    )

Acurácia média: 0.848899425368025
Desvio padrão da acurácia: 0.01282344029578005
Class: Nada
Precision		Recall			F1-Score		Support
0.8700 ± 0.0050		0.8587 ± 0.0238		0.8642 ± 0.0128		9154
Class: Grito
Precision		Recall			F1-Score		Support
0.6662 ± 0.0162		0.7739 ± 0.0161		0.7158 ± 0.0098		1457
Class: Violência Física
Precision		Recall			F1-Score		Support
0.8655 ± 0.0292		0.8614 ± 0.0079		0.8632 ± 0.0154		6124
Class: Tiro
Precision		Recall			F1-Score		Support
0.8701 ± 0.0152		0.8037 ± 0.0178		0.8352 ± 0.0060		1259
